In [ ]:

import io

class Mat:
    """Implementsção de uama matris
    """
    def __init__(self, rows, cols):
        #print(f"construtor do Mat invocado com {rows} e {cols}")
        self.mat = [[0 for c in range(cols)]
                    for r in range(rows)]
    def numRows (self): return len(self.mat)
    def numCols (self): return len(self.mat[0])
    def __str__(self):
        "Devolve a matriz como uma string"
        return '\n'.join(' '.join(str(val) for val in row)
                         for row in self.mat)
    def __repr__(self):
        "Utilizado no repl quando se pede para ver a matriz"
        return str(self)
    def __getitem__ (self, n):
        "Interface para a indexação []"
        return self.mat[n]

"""
2 chars iguais -> +3
diferentes     -> -1
"""

def subst(x, y):
  return 3 if x == y else -1
  
class NW:
  """
  Alinhamento global entre duas sequências
  """
  def __init__(self, s1, s2, g = -4):
    self.s1 = s1
    self.s2 = s2
    self.mat = Mat(len(s1) + 1, len(s2) + 1)
    self.tr  = Mat(len(s1) + 1, len(s2) + 1)
    
    for L in range(len(s1)):
      self.mat[L + 1][0] = g * (L + 1)
      self.tr[L + 1][0]  = 'C'

    for C in range(len(s2)):
      self.mat[0][C + 1] = g * (C + 1)
      self.tr[0][C + 1]  = 'E'

    for L, x1 in enumerate(s1):
      for C, x2 in enumerate(s2):
        possiveis = [
            self.mat[L  ][C    ] + subst(x1, x2),   # Diagonal
            self.mat[L+1][C    ] + g,               # Esquerda
            self.mat[L  ][C + 1] + g,               # Cima
        ]
        dirs = "DEC"

        self.mat[L + 1][C + 1] = max(possiveis)
        self.tr[L + 1][C + 1] = dirs[possiveis.index(self.mat[L + 1][C + 1])]

  def rebuild(self):
    # Começar no canto inferior direito
    L = len(self.s1)
    C = len(self.s2)
    S1 = ""
    S2 = ""
    
    dirs = {
        'D' : (-1, -1),
        'E' : ( 0, -1),
        'C' : (-1,  0)
    }

    while L > 0 or C > 0:
      #print(self.s1[L - 1], self.s2[C - 1])
      #print(L, C)
      #print(self.tr[L][C])
      DL, DC = dirs[self.tr[L][C]]

      if self.tr[L][C] == "D":
        S1 = self.s1[L - 1] + S1
        S2 = self.s2[C - 1] + S2
      elif self.tr[L][C] == "E":
        S1 = '-' + S1
        S2 = self.s2[C - 1] + S2
      else:
        S1 = self.s1[L - 1] + S1
        S2 = '-' + S2        

      L += DL
      C += DC

    return S1, S2

  def __repr__(self):
    cols = "-" + self.s2
    lins = "-" + self.s1
    with io.StringIO("") as S:
      print(' ', *cols, sep = '   ', file = S)
      for L, linha in zip(lins, self.mat):
        print(L, *[f'{x:3d}' for x in linha], file = S)

      print(file = S)

      print(' ', *cols, file = S)
      for L, linha in zip(lins, self.tr):
        print(L, *linha, file = S)

      return S.getvalue()
    
class SW(NW):
  """
  Alinhamento local entre duas sequências
  """
  def __init__(self, s1, s2, g = -4):
      self.s1 = s1
      self.s2 = s2
      self.mat = Mat(len(s1) + 1, len(s2) + 1)

      for L, x1 in enumerate(s1):
        for C, x2 in enumerate(s2):
          self.mat[L + 1][C + 1] = max(
              self.mat[L  ][C    ] + subst(x1, x2),
              self.mat[L+1][C    ] + g,
              self.mat[L  ][C + 1] + g,
              0
          )

In [ ]:
def best(*cars):
  return max(cars, key = lambda x : cars.count(x))

def consensus(*seqs):
  return ''.join([best(*s) for s in zip(*seqs)])


In [ ]:
s1,s2,s3 = "ATAGC AACC ATGAC".split()

a1, a2 = NW(s1, s2, g = -1).rebuild()

m1 = consensus(a1,a2)

m2, a3 = NW(m1, s3, g = -1).rebuild()

#print(m2,a3, sep="\n")

print(NW(s1, m2, g = -1).rebuild()[0])
print(NW(s2, m2, g = -1).rebuild()[0])
print(NW(s3, m2, g = -1).rebuild()[0])

In [ ]:
def consensus(s1, s2):
  res = ""
  for x, y in zip(s1, s2):
    if x == y:
      res += x
    elif x == '-':
      res += y
    else:
      res += x
  return res

In [ ]:
enzimas = {"R" : "[GA]", "Y" : "[CT]", "M" : "[AC]", "K" : "[GT]", "S" : "[GC]", "W" : "[AT]", "B" : "[^A]", "D" : "[^C]", "H" : "[^G]", "V" : "[^T]", "N" : "[ACGT]"}

enzima = "G^AMTV"

import re

def converte(enz):
  return ''.join([enzimas.get(x, x) for x in enz])

def enz_corte(enzima):
  e1, e2 = enzima.split('^')
  return f"(?<={converte(e1)}){converte(e2)}"

seq = 'TAAGACTGAATGACTCG'
pos = [m.span()[0] for m in re.finditer(enz_corte(enzima), seq)]

[seq[i:j] for i, j in zip([0, *pos], [*pos, len(seq)])]

In [ ]:
enz_corte("MYT^NVWAC")